In [267]:
import sys
sys.setrecursionlimit(10000) # 10000 is an example, try with different values
import geohash
import pandas as pd
import configparser
import googlemaps
import time
import json
import os
import networkx as nx
from networkx.readwrite import json_graph
from bs4 import BeautifulSoup
import requests
import csv

In [4]:
BASE_32 = "0123456789bcdefghjkmnpqrstuvwxyz"

NEIGHBORS = {'right':{'even': "bc01fg45238967deuvhjyznpkmstqrwx",'odd':"238967debc01fg45kmstqrwxuvhjyznp"},
             'left':{'even':"238967debc01fg45kmstqrwxuvhjyznp",'odd':"14365h7k9dcfesgujnmqp0r2twvyx8zb"},
             'top':{'even':"p0r21436x8zb9dcf5h7kjnmqesgutwvy",'odd':"bc01fg45238967deuvhjyznpkmstqrwx"},
             'bottom':{'even':"14365h7k9dcfesgujnmqp0r2twvyx8zb",'odd':"238967debc01fg45kmstqrwxuvhjyznp"}
            }
             
BORDERS = {'right':{'even':"bcfguvyz",'odd':"prxz"},
           'left':{'even':"0145hjnp",'odd':"028b"},
           'top':{'even':"prxz",'odd':"bcfguvyz"},
           'bottom':{'even':"028b",'odd':"0145hjnp"}
          }

def calc_adj_geohash(src_hash, search_direction):
    L_src_hash = src_hash.lower()
    last_char = L_src_hash[-1]
    hash_type = odd_even(L_src_hash)
    base = L_src_hash[0:len(L_src_hash)-1]
    if  last_char in BORDERS[search_direction][hash_type]:
        base = calc_adj_geohash(base, search_direction)
    return base + BASE_32[NEIGHBORS[search_direction][hash_type].index(last_char)]

In [5]:
def get_adj_geohashes(src_hash):
    directions = ['right','left','top','bottom']
    adj_hash_dict = {direction:calc_adj_geohash(src_hash,direction) for direction in directions}
    adj_hash_dict['top_right'] = calc_adj_geohash(adj_hash_dict['right'],'top')
    adj_hash_dict['bottom_right'] = calc_adj_geohash(adj_hash_dict['right'],'bottom')
    adj_hash_dict['top_left'] = calc_adj_geohash(adj_hash_dict['left'],'top')
    adj_hash_dict['bottom_left'] = calc_adj_geohash(adj_hash_dict['left'],'bottom')
    adj_hash_dict['middle'] = src_hash
    return adj_hash_dict    

In [6]:
def odd_even(str_hash):
    if len(str_hash)%2 == 0:
        return 'even'
    else:
        return 'odd'

In [7]:
config = configparser.ConfigParser()
config.read("config.ini")
API_key = config['Keys']['google_API']
gmaps = googlemaps.Client(key=API_key)

def get_geohash_distance(gh_A,gh_B):   
    GPS_A = geohash.decode(gh_A)
    GPS_B = geohash.decode(gh_B) 
    directions_result = gmaps.directions(GPS_A,
                                         GPS_B,
                                         mode="driving")
    time.sleep(1)
    return directions_result[0]['legs'][0]['distance']['value'] 

In [274]:
GEOHASH_PRECISION = 3
MAX_RANGE = 426 #Maximum Tesla Model S range in Km from https://en.wikipedia.org/wiki/Tesla_Model_S
POP_DICT = load_pop_dict()
def build_connections(G,src_hash):
    print (nx.number_of_nodes(G))
    connections = {}
    node_hashes = ([node for node in G
                  if node[0:GEOHASH_PRECISION] in list(get_adj_geohashes(src_hash[0:GEOHASH_PRECISION]).values())
                  and node != src_hash])
    close_connections = ([{'node':node_gh,
                           'distance':get_geohash_distance(src_hash,node_gh)} for node_gh in node_hashes])
    for connection in close_connections:
        if connection['distance']/1000 <= MAX_RANGE:
            edge_weight = get_edge_weight(G,src_hash,connection['node'])
            G.add_edge(src_hash,connection['node'],{'weight':edge_weight,'distance':connection['distance'],
                                                    'lon_lat_1':reverse_GPS(geohash.decode(src_hash)),
                                                    'lon_lat_2':reverse_GPS(geohash.decode(connection['node']))})
    return G

In [273]:
def get_edge_weight(G,src_hash,connection_node):
    try:
        pop1 = POP_DICT[G.node[src_hash]['city_state']]
        pop2 = POP_DICT[G.node[connection_node]['city_state']]
        return (pop1+pop2)/POP_DICT['Total']
    except KeyError as e:
        print(e)
        return 0    

In [9]:
def reverse_GPS(GPS):
    return [GPS[1],GPS[0]]

In [276]:
def build_network():
    G = load_network()
    df = pd.read_csv("Teslarati_SC_data.csv",dtype={'Stalls': float,'Zip':str,'Tesla':str,'Elev':str})
    df["lat"], df["lon"] = zip(*df["GPS"].str.split(',').tolist())
    df["lat"], df["lon"] = df["lat"].astype(float), df["lon"].astype(float)
    df['GPS_lon_lat'] = df.apply(lambda x: [x["lon"],x["lat"]], axis=1)
    df['geohash'] = df.apply(lambda x: geohash.encode(x['lat'],x['lon']), axis=1)
    df['SC_data'] = df.apply(lambda x: parse_Tesla_SC_data(x['Tesla']), axis=1)
    
    for i in df['geohash'].keys():
        if df['geohash'][i] not in G:
            G.add_node(df['geohash'][i],{key:df[key][i] for key in df.keys()})
            build_connections(G,df['geohash'][i])
    
    full_network = json_graph.node_link_data(G)
    slim_network = json_graph.node_link_data(trim_network(G))
    with open("full_network.json","w") as f:
        json.dump(full_network,f)
    with open("slim_network.json","w") as f:
        json.dump(slim_network,f)
    return G

In [185]:
def load_network():
    if os.path.getsize("full_network.json") > 0:
        with open("full_network.json","r") as f:
            data = json.load(f)
            G = json_graph.node_link_graph(data)
    else:
        G=nx.Graph()
    return G

In [252]:
def load_pop_dict():
    p_dict = {}
    with open('populations.csv', 'r') as f:
        data = csv.DictReader(f)
        for row in data:
            p_dict[row['City_State']] = int(row['Population'])
    return p_dict

In [ ]:
test_G = build_network()

In [183]:
def parse_Tesla_SC_data(URL):
    SC_data = {}
    r = requests.get(URL)
    soup = BeautifulSoup(r.text,"html.parser")
    attr_lists = soup.find_all('p')
    for attr in attr_lists:
        if attr.find('strong'):
            #probably not ideal, but only way I could get BS to parse 'br' correctly. Better way must exist.
            SC_data[attr.find('strong').text] = [value for value in attr.childGenerator()
                                                 if value.name == None and value != ' ']
    time.sleep(1)
    return SC_data

In [271]:
def trim_network(G):
    slim_G = G.copy()
    for node in slim_G:
        del (slim_G.node[node]['SC_data'])
    return slim_G

In [145]:
parse_Tesla_SC_data('http://www.teslamotors.com/findus/location/supercharger/barstowsupercharger')

{'Amenities': ['Tanger Outlet mall, take the foot path around the AM/PM'],
 'Charging': ['8 Supercharger stalls, available 24/7'],
 'Public Charging': ['Pay NEMA 14-50 Outlet',
  'Barstow Calico KOA',
  '35250 Outer Highway 15 North',
  'Yermo, CA 92398'],
 'Restrooms': ["Chili's", 'Country Inn (open 24hrs)'],
 'Wifi': ['Country Inn',
  'Starbucks (Starbucks is located behind Country Inn, not visible from the charging stalls)']}